In [2]:
import yfinance as yf
import pandas as pd

tickers = [
    "EWRK.ST","NOVO-B.CO","FOUR","BTS-B.ST","NWG.ST","EVO.ST","BETS-B.ST",
    "VOLV-B.ST","ERIC-B.ST","SAND.ST","ATCO-A.ST","HM-B.ST","INVE-B.ST",
    "SWED-A.ST","SHB-A.ST","SKF-B.ST","ASSA-B.ST","SEB-A.ST","TEL2-B.ST","SCA-B.ST",
    "INDU-C.ST","LATO-B.ST","BALD-B.ST","CAST.ST","HEXP.ST","KINV-B.ST",
    "ALFA.ST","LUN.ST","NIBE-B.ST","SSAB-A.ST"
]

rows = []
for t in tickers:
    try:
        stock = yf.Ticker(t)
        info = stock.info
        hist = stock.history(period="1d")

        rows.append({
            "Ticker": t,
            "Price": hist["Close"].iloc[-1] if not hist.empty else None,
            "MarketCap": info.get("marketCap"),
            "PE": info.get("trailingPE"),
            "PB": info.get("priceToBook"),
            "EV/EBITDA": info.get("enterpriseToEbitda"),
            "ROE": info.get("returnOnEquity"),
            "Debt/Equity": info.get("debtToEquity"),
            "Revenue": info.get("totalRevenue"),
            "NetIncome": info.get("netIncomeToCommon"),
            "OperatingMargin": info.get("operatingMargins"),
            "GrossMargins": info.get("grossMargins"),
            "FreeCashflow": info.get("freeCashflow"),
            "Beta": info.get("beta"),
            "DividendYield": info.get("dividendYield")
        })
    except Exception as e:
        print(f"Fel med {t}: {e}")

# Skapa DataFrame
df = pd.DataFrame(rows)

# Ta bort .ST från tickers
df["Ticker"] = df["Ticker"].str.replace(".ST", "", regex=False)

# Formatera siffror
numeric_cols_int = ["MarketCap", "Revenue", "NetIncome", "FreeCashflow"]  # inga decimaler, tusentalsavgränsning
numeric_cols_round = ["PE", "PB", "EV/EBITDA", "ROE", "Debt/Equity", "OperatingMargin", "GrossMargins", "Beta", "DividendYield"]

# Rensa bort None och NaN
df = df.fillna(0)

# Formatera stora tal med tusentalsavgränsning
for col in numeric_cols_int:
    df[col] = df[col].astype(float).round(0).astype(int).apply(lambda x: f"{x:,}".replace(",", " "))

# Runda nyckeltal till heltal
for col in numeric_cols_round:
    df[col] = df[col].astype(float).round(0).astype(int)

# Spara ren fil
df.to_csv("fundamental_data.csv", index=False, encoding="utf-8-sig")

print("✅ Data tvättad och sparad till fundamental_data.csv")
print(df.head())


$NWG.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: HEXP.ST"}}}
$HEXP.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$LUN.ST: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


✅ Data tvättad och sparad till fundamental_data.csv
      Ticker       Price          MarketCap  PE  PB  EV/EBITDA  ROE  \
0       EWRK  107.000000      1 849 738 496  16   8         12    1   
1  NOVO-B.CO  363.450012  1 614 964 588 544  15  10         11    1   
2       FOUR   75.510002      6 677 433 856  28   8         12    0   
3      BTS-B  188.399994      3 654 360 832  11   2         11    0   
4        NWG    0.000000                  0   0   0          0    0   

   Debt/Equity          Revenue        NetIncome  OperatingMargin  \
0          116   14 499 482 624      117 594 000                0   
1           59  311 937 990 656  111 067 996 160                0   
2          187    3 610 700 032      207 700 000                0   
3           34    2 821 441 024      337 822 016                0   
4            0                0                0                0   

   GrossMargins    FreeCashflow  Beta  DividendYield  
0             0     212 415 120     0              